In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install numpy==1.19.4
!pip install tensorflow==2.4.0
!pip install tensorflow-hub==0.10.0
!pip install bert-for-tf2==0.14.7
!pip install sentencepiece==0.1.94

     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30537 sha256=f31f187ee2b2d3e4c209b920070c23056c8d87f4e1a71077be19bddb497651d9
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7303 sha256=7f32533d8d4f52000183119b0383b92e5776726a28554fed2be12868c168e164
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19474 sha256=6e7a9ccad40107f7f787ece06a7003abccd515fd9c66495415a6fa8cbf55cdf6
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 8.1MB/s 


In [3]:
import numpy as np

try:
    %tensorflow_version 2.x
except Exception:
    pass

import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [4]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [13]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, '[PAD]').astype(int)

def get_segments(tokens):
    current_seg_id, seg_ids = 0, []
    for token in tokens:
        seg_ids.append(current_seg_id)
        if token == '[SEP]':
            current_seg_id = 1-current_seg_id
    return seg_ids

In [19]:
old_model = tf.keras.models.load_model('/content/drive/MyDrive/Text-Classification/model')
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Text-Classification-v1/model')

In [23]:
def get_predictions_old(model, sentence):

    def encode_sentence(sentence):
        return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence))

    if len(sentence.split()) < 3: return 'neutral'
    tokens = encode_sentence(sentence)
    inputs = tf.expand_dims(tokens, 0)
    output = model(inputs, training=False)
    return "negative" if not np.floor(output*2) else "positive"

In [18]:
def get_predictions_new(model, sentence):

    def encode_sentence(sentence):
        return ['[CLS]']+tokenizer.tokenize(sentence)+['[SEP]']

    tokens = encode_sentence(sentence)
    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)
    inputs = tf.stack([tf.cast(input_ids, dtype=tf.int32), tf.cast(input_mask, dtype=tf.int32), tf.cast(segment_ids, dtype=tf.int32)], axis=0)
    inputs = tf.expand_dims(inputs, 0)
    output = model(inputs, training=False)
    return "negative" if not np.floor(output*2) else "positive"

In [26]:
print(get_predictions_old(old_model, 'today is my favourite day'))
print(get_predictions_old(old_model, 'today is not a awful day'))

positive
negative


In [28]:
print(get_predictions_new(model, 'today is my favourite day'))
print(get_predictions_new(model, 'today is not my favourite day'))

positive
negative
